# Preprocessing Median and 90th Percentile Hs Fields
This script outputs ensemble and time averaged median and 90th percentile fields for a specific month.
The process is run in for loop below iterating over months jul, sep, nov and time slices 2020-2040, 2050-2070

In [3]:
import xarray as xr
import numpy as np
import os

### Functions for reading WW3 simulation data

In [4]:
def find_ww3_file(root_directory,year):
    for root, dirs, files in os.walk(root_directory):
        for file in files:
            if str(year) in file:
                matching_file = (os.path.join(root, file))
    return matching_file

def extract_ww3_data(ncfile,var,year,month):
    with xr.open_dataset(ncfile) as ds:
        ds = ds.sel(time=ds.time.dt.month.isin([month]))
        dt = np.array(ds.coords['time'][:])        
        x,y = np.array(ds.variables['longitude'][:]),np.array(ds.variables['latitude'][:])
        var_field = np.array(ds.variables[var][:])
    return var_field,x,y,dt

def concatenate_ww3(rootdir,var,year0,yearn,month):
    if var == 'hs':
        rootvardir = rootdir + var
    elif var == 'tp':
        rootvardir = rootdir + var
    years = np.arange(year0,yearn,1)
    for count,year in enumerate(years):
        file = find_ww3_file(rootvardir,year)
        temp,x,y,temp_dt = extract_ww3_data(file,var,year,month)
        if count == 0:
            series = temp
            dt = temp_dt
        else:
            series = np.vstack((series,temp))
            dt = np.append(dt,temp_dt)
    series = np.where(np.isnan(series),0,series)
    return series,dt

def wavefield_stats(hs,percentile):
    hs_med = np.nanmedian(hs,axis =0)
    hs_per = np.percentile(hs, percentile,axis = 0) 
    return hs_med, hs_per

### Derive and save ensemble and yearly averaged monthly 50th/90th percentile Hs fields

In [7]:
months = [7,9,11]
year0s = [2020,2050]
yearns = [2040,2070]

for year0,yearn in zip(year0s,yearns):
    for month in months:
        output_filename = f'WW3_{year0}_{yearn}_{month}.nc'
        print(output_filename)

        #CNRM
        rootdir = '/WW3/CNRM/'
        #Load the Hs data 
        hs,dt = concatenate_ww3(rootdir,'hs',year0,yearn,month)
        hs_med_cnrm, hs_per_cnrm = wavefield_stats(hs,90)

        #ECEARTH
        rootdir = '/WW3/ECEARTH/'
        #Load the Hs data 
        hs,dt = concatenate_ww3(rootdir,'hs',year0,yearn,month)
        hs_med_ecearth, hs_per_ecearth = wavefield_stats(hs,90)

        #MPI
        rootdir = '/WW3/MPI/'
        #Load the Hs data 
        hs,dt = concatenate_ww3(rootdir,'hs',year0,yearn,month)
        hs_med_mpi, hs_per_mpi = wavefield_stats(hs,90)

        #MRI
        rootdir = '/WW3/MRI/'
        #Load the Hs data 
        hs,dt = concatenate_ww3(rootdir,'hs',year0,yearn,month)
        hs_med_mri, hs_per_mri = wavefield_stats(hs,90)
        
        ave_per = np.array([hs_per_cnrm,hs_per_ecearth,hs_per_mpi,hs_per_mri])
        ave_per = np.nanmean(ave_per,axis =0)
        
        ave_med = np.array([hs_med_cnrm,hs_med_ecearth,hs_med_mpi,hs_med_mri])
        ave_med = np.nanmean(ave_med,axis =0)
        
        nodes = ave_med.shape[0]

        hs_med_array = xr.DataArray(ave_med,
                                    dims=('nodes'),
                                    coords={'nodes': np.arange(nodes)},
                                    name='hs_med')

        hs_per_array = xr.DataArray(ave_per,
                                    dims=('nodes'),
                                    coords={'nodes': np.arange(nodes)},
                                    name='hs_per')


        dataset = xr.Dataset({'hs_med': hs_med_array, 'hs_per': hs_per_array})
        dataset.to_netcdf(output_filename)
        print(f"NetCDF file '{output_filename}' saved successfully.")


WW3_2020_2040_7.nc
NetCDF file 'WW3_2020_2040_7.nc' saved successfully.
WW3_2020_2040_9.nc
NetCDF file 'WW3_2020_2040_9.nc' saved successfully.
WW3_2020_2040_11.nc
NetCDF file 'WW3_2020_2040_11.nc' saved successfully.
WW3_2050_2070_7.nc
NetCDF file 'WW3_2050_2070_7.nc' saved successfully.
WW3_2050_2070_9.nc
NetCDF file 'WW3_2050_2070_9.nc' saved successfully.
WW3_2050_2070_11.nc
NetCDF file 'WW3_2050_2070_11.nc' saved successfully.
